## Generic Types:

1. Abstract Types
2. Custom Types
3. Mutable Types
4. Case study: auto-diff.

## Benchmarking and Profiling
1. Benchmark Tools
2. Profiling Tools

## Performant Code

1. Data Access Patterns
2. Looped Code vs Broadcast: example: sum every second element
3. @simd, @inbounds
4. Code mutation

In [15]:
a = rand(10000, 10000);
using BenchmarkTools

In [13]:
tmp1 = 0;
@time for j = 1:size(a)[2]
    for i = size(a)[1]
        tmp1+=a[i,j]
    end
end

  0.006864 seconds (88.98 k allocations: 1.663 MiB)


In [14]:
tmp2 = 0;
@time for i = 1:size(a)[1]
    for j = size(a)[2]
        tmp2+=a[i,j]
    end
end

  0.005012 seconds (88.98 k allocations: 1.663 MiB)


## Functional Programming

What is it? Why bother?

1. Map
2. Filter
3. Apply
4. Reduce